In [0]:
from pyspark.sql import SparkSession


In [0]:
#create session
spark = SparkSession.builder.appName("T20 Data Analysis").getOrCreate()

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
fact_bating = spark.read.format("csv").option("header", "true").load('s3://t20worldcup/fact_bating/fact_bating.csv')


In [0]:
fact_bating = fact_bating.withColumnRenamed("out/not_out", "Innings")


In [0]:
fact_bating.show()

+--------------------+-----------+----------+--------------------+----+-----+---+---+------+-------+-----------+
|               match|teamInnings|battingPos|         batsmanName|runs|balls| 4s| 6s|    SR|Innings|   match_id|
+--------------------+-----------+----------+--------------------+----+-----+---+---+------+-------+-----------+
|Namibia Vs Sri Lanka|    Namibia|         1|  Michael van Lingen|   3|    6|  0|  0| 50.00|    out|T20I # 1823|
|Namibia Vs Sri Lanka|    Namibia|         2|       Divan la Cock|   9|    9|  1|  0|100.00|    out|T20I # 1823|
|Namibia Vs Sri Lanka|    Namibia|         3|Jan Nicol Loftie-...|  20|   12|  1|  2|166.66|    out|T20I # 1823|
|Namibia Vs Sri Lanka|    Namibia|         4|       Stephan Baard|  26|   24|  2|  0|108.33|    out|T20I # 1823|
|Namibia Vs Sri Lanka|    Namibia|         5|  Gerhard Erasmus(c)|  20|   24|  0|  0| 83.33|    out|T20I # 1823|
|Namibia Vs Sri Lanka|    Namibia|         6|        Jan Frylinck|  44|   28|  4|  0|157.14|    

In [0]:
fact_bowling = spark.read.format("csv").option("header", "true").load('s3://t20worldcup/fact_bowling/fact_bowling_summary.csv')


In [0]:
fact_bowling.show()

+--------------------+-----------+--------------------+-----+------+----+-------+-------+---+---+---+-----+-------+-----------+-------+-----------+
|               match|bowlingTeam|          bowlerName|overs|maiden|runs|wickets|economy| 0s| 4s| 6s|wides|noBalls|   match_id|subover|bowlsbowled|
+--------------------+-----------+--------------------+-----+------+----+-------+-------+---+---+---+-----+-------+-----------+-------+-----------+
|Namibia Vs Sri Lanka|  Sri Lanka|  Maheesh Theekshana|    4|     0|  23|      1|   5.75|  7|  0|  0|    2|      0|T20I # 1823|      0|         24|
|Namibia Vs Sri Lanka|  Sri Lanka| Dushmantha Chameera|    4|     0|  39|      1|   9.75|  6|  3|  1|    2|      0|T20I # 1823|      0|         24|
|Namibia Vs Sri Lanka|  Sri Lanka|     Pramod Madushan|    4|     0|  37|      2|   9.25|  6|  3|  1|    0|      0|T20I # 1823|      0|         24|
|Namibia Vs Sri Lanka|  Sri Lanka| Chamika Karunaratne|    4|     0|  36|      1|      9|  7|  3|  1|    1|     

In [0]:
match_summary = spark.read.format("csv").option("header", "true").load('s3://t20worldcup/match_summary/match_summary.csv')


In [0]:
match_summary.show()

+------------+-----------+-----------+---------+---------+------------+-----------+
|       team1|      team2|     winner|   margin|   ground|   matchDate|   match_id|
+------------+-----------+-----------+---------+---------+------------+-----------+
|     Namibia|  Sri Lanka|    Namibia|  55 runs|  Geelong|Oct 16, 2022|T20I # 1823|
| Netherlands|     U.A.E.|Netherlands|3 wickets|  Geelong|Oct 16, 2022|T20I # 1825|
|    Scotland|West Indies|   Scotland|  42 runs|   Hobart|Oct 17, 2022|T20I # 1826|
|     Ireland|   Zimbabwe|   Zimbabwe|  31 runs|   Hobart|Oct 17, 2022|T20I # 1828|
|     Namibia|Netherlands|Netherlands|5 wickets|  Geelong|Oct 18, 2022|T20I # 1830|
|   Sri Lanka|     U.A.E.|  Sri Lanka|  79 runs|  Geelong|Oct 18, 2022|T20I # 1832|
|     Ireland|   Scotland|    Ireland|6 wickets|   Hobart|Oct 19, 2022|T20I # 1833|
| West Indies|   Zimbabwe|West Indies|  31 runs|   Hobart|Oct 19, 2022|T20I # 1834|
| Netherlands|  Sri Lanka|  Sri Lanka|  16 runs|  Geelong|Oct 20, 2022|T20I 

In [0]:
players = spark.read.format("csv").option("header", "true").load('s3://t20worldcup/players/players.csv')


In [0]:
players.show()

+--------------------+----------+--------------------+--------------+--------------------+-------------------+--------------------+
|                name|      team|               image|  battingStyle|        bowlingStyle|        playingRole|         description|
+--------------------+----------+--------------------+--------------+--------------------+-------------------+--------------------+
|Najmul Hossain Sh...|Bangladesh|                null| Left hand Bat|  Right arm Offbreak|   Top order Batter|Nazmul Hossain Sh...|
|       Soumya Sarkar|Bangladesh|                null| Left hand Bat|Right arm Medium ...|Middle order Batter|A rarity among Ba...|
|          Litton Das|Bangladesh|                null|Right hand Bat|                null|Wicketkeeper Batter|Liton Das is the ...|
|  Shakib Al Hasan(c)|Bangladesh|                null| Left hand Bat|Slow Left arm Ort...|         Allrounder|When the annals o...|
|        Afif Hossain|Bangladesh|                null| Left hand Bat|  Right

# Transformation


In [0]:
players.createOrReplaceTempView("players")
match_summary.createOrReplaceTempView("match_summary")
fact_bowling.createOrReplaceTempView("fact_bowling")
fact_bating.createOrReplaceTempView("fact_bating")


In [0]:
from pyspark.sql.functions import col, sum, count, when ,round 



In [0]:
openers = fact_bating.groupBy("batsmanName", "battingPos").agg(
    sum(col("runs")).alias("Total_run"),
    count("match_id").alias("Innings_Batted"),
    sum("balls").alias("Total_balls"),
    round(sum("runs") / sum("balls") * 100).alias("StrikeRate"),
    count(when(col("Innings") == "out", 1)).alias("total_innings_Dismissed"),
    round(sum("balls") / count("match_id")).alias("Avg_balls_Faced"),
    round(sum(col("runs")) / count(when(col("Innings") == "out", 1))).alias("Batting_Average"),
    round(((sum("4s") * 4) + (sum("6s") * 6)) / sum(col("runs")) * 100).alias("battingper"),
).orderBy(col("Total_run").desc())

openers.show(2)


+----------------+----------+---------+--------------+-----------+----------+-----------------------+---------------+---------------+----------+
|     batsmanName|battingPos|Total_run|Innings_Batted|Total_balls|StrikeRate|total_innings_Dismissed|Avg_balls_Faced|Batting_Average|battingper|
+----------------+----------+---------+--------------+-----------+----------+-----------------------+---------------+---------------+----------+
|     Virat Kohli|         3|    296.0|             6|      217.0|     136.0|                      3|           36.0|           99.0|      50.0|
|Suryakumar Yadav|         4|    239.0|             6|      126.0|     190.0|                      4|           21.0|           60.0|      66.0|
+----------------+----------+---------+--------------+-----------+----------+-----------------------+---------------+---------------+----------+
only showing top 2 rows



In [0]:
bowling = spark.read.format("csv").option("inferscheme",True).option("header",True).option("sep",",").load('s3://t20worldcup/fact_bowling/fact_bowling_summary.csv')
bowling.show()

+--------------------+-----------+--------------------+-----+------+----+-------+-------+---+---+---+-----+-------+-----------+-------+-----------+
|               match|bowlingTeam|          bowlerName|overs|maiden|runs|wickets|economy| 0s| 4s| 6s|wides|noBalls|   match_id|subover|bowlsbowled|
+--------------------+-----------+--------------------+-----+------+----+-------+-------+---+---+---+-----+-------+-----------+-------+-----------+
|Namibia Vs Sri Lanka|  Sri Lanka|  Maheesh Theekshana|    4|     0|  23|      1|   5.75|  7|  0|  0|    2|      0|T20I # 1823|      0|         24|
|Namibia Vs Sri Lanka|  Sri Lanka| Dushmantha Chameera|    4|     0|  39|      1|   9.75|  6|  3|  1|    2|      0|T20I # 1823|      0|         24|
|Namibia Vs Sri Lanka|  Sri Lanka|     Pramod Madushan|    4|     0|  37|      2|   9.25|  6|  3|  1|    0|      0|T20I # 1823|      0|         24|
|Namibia Vs Sri Lanka|  Sri Lanka| Chamika Karunaratne|    4|     0|  36|      1|      9|  7|  3|  1|    1|     

In [0]:
Bowler = bowling.groupBy("bowlerName").agg(
    sum("wickets").alias("Total_wickets"),
    sum("bowlsbowled").alias("balls_Bowled"),
    sum("runs").alias("Runs_Conceded"),
    round(col("Runs_Conceded") / (col("balls_Bowled") / 6)).alias("economy"),
    round(col("balls_Bowled") / col("Total_wickets")).alias("Bowling_Strike_Rate"),
    round(col("Runs_Conceded") / col("Total_wickets")).alias("Bowling_Average"),
    (count("match_id")).alias("Innings_Bowled"),
    round(sum("0s") / sum("bowlsbowled")* 100).alias("bowlingper")
).orderBy(col("Total_wickets").desc())

Bowler.show(2)

+--------------------+-------------+------------+-------------+-------+-------------------+---------------+--------------+----------+
|          bowlerName|Total_wickets|balls_Bowled|Runs_Conceded|economy|Bowling_Strike_Rate|Bowling_Average|Innings_Bowled|bowlingper|
+--------------------+-------------+------------+-------------+-------+-------------------+---------------+--------------+----------+
|Wanindu Hasaranga...|         15.0|       186.0|        199.0|    6.0|               12.0|           13.0|             8|      40.0|
|          Sam Curran|         13.0|       136.0|        148.0|    7.0|               10.0|           11.0|             6|      49.0|
+--------------------+-------------+------------+-------------+-------+-------------------+---------------+--------------+----------+
only showing top 2 rows



In [0]:
top_order = openers.filter(
    (openers["Batting_Average"] > 30)
    & (openers["StrikeRate"] > 140)
    & (openers["Innings_Batted"] > 3)
    & (openers["battingper"] > 50)
    & (openers["battingPos"] < 4)
)

# Show the resulting DataFrame
top_order.show()

+--------------+----------+---------+--------------+-----------+----------+-----------------------+---------------+---------------+----------+
|   batsmanName|battingPos|Total_run|Innings_Batted|Total_balls|StrikeRate|total_innings_Dismissed|Avg_balls_Faced|Batting_Average|battingper|
+--------------+----------+---------+--------------+-----------+----------+-----------------------+---------------+---------------+----------+
|Jos Buttler(c)|         1|    225.0|             6|      156.0|     144.0|                      5|           26.0|           45.0|      61.0|
|    Alex Hales|         2|    212.0|             6|      144.0|     147.0|                      5|           24.0|           42.0|      64.0|
| Rilee Rossouw|         3|    141.0|             4|       83.0|     170.0|                      4|           21.0|           35.0|      64.0|
+--------------+----------+---------+--------------+-----------+----------+-----------------------+---------------+---------------+----------+

In [0]:
middle_order = openers.filter(
    (openers["Batting_Average"] > 40)
    & (openers["StrikeRate"] > 125)
    & (openers["Innings_Batted"] > 3)
    & (openers["Avg_balls_Faced"] > 20)
    & (openers["battingPos"] > 2)
)
middle_order.show()

+----------------+----------+---------+--------------+-----------+----------+-----------------------+---------------+---------------+----------+
|     batsmanName|battingPos|Total_run|Innings_Batted|Total_balls|StrikeRate|total_innings_Dismissed|Avg_balls_Faced|Batting_Average|battingper|
+----------------+----------+---------+--------------+-----------+----------+-----------------------+---------------+---------------+----------+
|     Virat Kohli|         3|    296.0|             6|      217.0|     136.0|                      3|           36.0|           99.0|      50.0|
|Suryakumar Yadav|         4|    239.0|             6|      126.0|     190.0|                      4|           21.0|           60.0|      66.0|
+----------------+----------+---------+--------------+-----------+----------+-----------------------+---------------+---------------+----------+



In [0]:
lower_order = openers.filter(
    (openers["Batting_Average"] > 25) &
    (openers["StrikeRate"] > 130) &
    (openers["Innings_Batted"] > 3) &
    (openers["Avg_balls_Faced"] > 12) &
    (openers["battingPos"] > 4)
).join(
    Bowler.filter(Bowler["Innings_Bowled"] > 1),
    openers["batsmanName"] == Bowler["bowlerName"]
).select(
    openers["batsmanName"],
    openers["Batting_Average"],
    openers["StrikeRate"],
    openers["Innings_Batted"],
    openers["Avg_balls_Faced"],
    openers["battingPos"],
    Bowler["Innings_Bowled"]
)
lower_order.show()

+--------------+---------------+----------+--------------+---------------+----------+--------------+
|   batsmanName|Batting_Average|StrikeRate|Innings_Batted|Avg_balls_Faced|battingPos|Innings_Bowled|
+--------------+---------------+----------+--------------+---------------+----------+--------------+
|Marcus Stoinis|           42.0|     162.0|             4|           20.0|         5|             4|
| Sikandar Raza|           31.0|     150.0|             7|           21.0|         5|             8|
|Curtis Campher|           30.0|     170.0|             5|           14.0|         5|             4|
+--------------+---------------+----------+--------------+---------------+----------+--------------+



In [0]:
Allrounder = openers.filter(
    (openers["Batting_Average"] > 15) &
    (openers["StrikeRate"] > 140) &
    (openers["Innings_Batted"] > 2) &
    (openers["battingPos"] > 4)
).join(
    Bowler.filter(
        (Bowler["Innings_Bowled"] > 2) &
        (Bowler["economy"] < 7) &
        (Bowler["Bowling_Strike_Rate"] < 20)
    ),
    openers["batsmanName"] == Bowler["bowlerName"]
).select(
    openers["batsmanName"],
    openers["Batting_Average"],
    openers["StrikeRate"],
    openers["Innings_Batted"],
    openers["battingPos"],
    Bowler["Innings_Bowled"],
    Bowler["economy"],
    Bowler["Bowling_Strike_Rate"]
)

# Show the result
Allrounder.show()

+-----------------+---------------+----------+--------------+----------+--------------+-------+-------------------+
|      batsmanName|Batting_Average|StrikeRate|Innings_Batted|battingPos|Innings_Bowled|economy|Bowling_Strike_Rate|
+-----------------+---------------+----------+--------------+----------+--------------+-------+-------------------+
|Paul van Meekeren|           23.0|     167.0|             3|        11|             8|    6.0|               17.0|
| Mitchell Santner|           27.0|     169.0|             3|         7|             5|    6.0|               13.0|
+-----------------+---------------+----------+--------------+----------+--------------+-------+-------------------+



In [0]:
fast_bowlers = Bowler.filter(
    (Bowler["Innings_Bowled"] > 4) &
    (Bowler["economy"] < 7) &
    (Bowler["Bowling_Strike_Rate"] < 16) &
    (Bowler["Bowling_Average"] < 20) &
    (Bowler["bowlingper"] > 40)
).join(
    players.filter(players["bowlingStyle"].like("%Fast%")),
    Bowler["bowlerName"] == players["name"]
).select(
    Bowler["bowlerName"],
    Bowler["Innings_Bowled"],
    Bowler["economy"],
    Bowler["Bowling_Strike_Rate"],
    Bowler["Bowling_Average"],
    players["bowlingStyle"],
    Bowler["bowlingper"]
)
fast_bowlers.show()

+-------------------+--------------+-------+-------------------+---------------+--------------+----------+
|         bowlerName|Innings_Bowled|economy|Bowling_Strike_Rate|Bowling_Average|  bowlingStyle|bowlingper|
+-------------------+--------------+-------+-------------------+---------------+--------------+----------+
|Shaheen Shah Afridi|             7|    6.0|               14.0|           14.0| Left arm Fast|      46.0|
|      Anrich Nortje|             5|    5.0|               10.0|            9.0|Right arm Fast|      55.0|
+-------------------+--------------+-------+-------------------+---------------+--------------+----------+



# SQL Calculation


In [0]:
openers.createOrReplaceTempView("openers")
Bowler.createOrReplaceTempView("Bowler")
players.createOrReplaceTempView("players")


In [0]:
%sql
select batsmanName,Batting_Average,StrikeRate,Innings_Batted, battingper , battingPos from openers
where Batting_Average > 30 and StrikeRate > 140 and Innings_Batted >3 and battingper>50 and battingPos< 4

batsmanName,Batting_Average,StrikeRate,Innings_Batted,battingper,battingPos
Jos Buttler(c),45.0,144.0,6,61.0,1
Alex Hales,42.0,147.0,6,64.0,2
Rilee Rossouw,35.0,170.0,4,64.0,3


In [0]:
%sql
select batsmanName,Batting_Average,StrikeRate,Innings_Batted, Avg_balls_Faced , battingPos from openers
where Batting_Average > 40 and StrikeRate > 125 and Innings_Batted >3 and Avg_balls_Faced > 20 and battingPos > 2

batsmanName,Batting_Average,StrikeRate,Innings_Batted,Avg_balls_Faced,battingPos
Virat Kohli,99.0,136.0,6,36.0,3
Suryakumar Yadav,60.0,190.0,6,21.0,4


In [0]:
%sql
select a.batsmanName,a.Batting_Average,a.StrikeRate,a.Innings_Batted, a.Avg_balls_Faced , a.battingPos, b.Innings_Bowled from openers as a
inner join Bowler as b on a.batsmanName = b.bowlerName
where a.Batting_Average > 25 and a.StrikeRate > 130 and a.Innings_Batted >3 and a.Avg_balls_Faced > 12 and a.battingPos > 4 and b.Innings_Bowled > 1

batsmanName,Batting_Average,StrikeRate,Innings_Batted,Avg_balls_Faced,battingPos,Innings_Bowled
Marcus Stoinis,42.0,162.0,4,20.0,5,4
Sikandar Raza,31.0,150.0,7,21.0,5,8
Curtis Campher,30.0,170.0,5,14.0,5,4


In [0]:
%sql
select a.batsmanName,a.Batting_Average,a.StrikeRate,a.Innings_Batted, a.battingPos, b.Innings_Bowled ,b.economy,b.Bowling_Strike_Rate from openers as a
inner join Bowler as b on a.batsmanName = b.bowlerName
where a.Batting_Average > 15 and a.StrikeRate > 140 and a.Innings_Batted >2  and a.battingPos > 4 and b.Innings_Bowled > 2 and b.economy< 7 and b.Bowling_Strike_Rate <20

batsmanName,Batting_Average,StrikeRate,Innings_Batted,battingPos,Innings_Bowled,economy,Bowling_Strike_Rate
Mitchell Santner,27.0,169.0,3,7,5,6.0,13.0
Paul van Meekeren,23.0,167.0,3,11,8,6.0,17.0


In [0]:
%sql
select a.bowlerName ,a.Innings_Bowled ,a.economy, a.Bowling_Strike_Rate, a.Bowling_Average, b.bowlingStyle, a.bowlingper 
from Bowler as a 
inner join players as b on a.bowlerName = b.name
where a.Innings_Bowled > 4 and a.economy < 7 and a.Bowling_Strike_Rate < 16 and a.Bowling_Average < 20 and b.bowlingStyle LIKE '%Fast%'
 and a.bowlingper > 40





bowlerName,Innings_Bowled,economy,Bowling_Strike_Rate,Bowling_Average,bowlingStyle,bowlingper
Shaheen Shah Afridi,7,6.0,14.0,14.0,Left arm Fast,46.0
Anrich Nortje,5,5.0,10.0,9.0,Right arm Fast,55.0


In [0]:

top_order_col = top_order.select(top_order.columns[0])
middle_order_col = middle_order.select(middle_order.columns[0])
lower_order_col = lower_order.select(lower_order.columns[0])
Allrounder_col = Allrounder.select(Allrounder.columns[0])
fast_bowlers_col = fast_bowlers.select(fast_bowlers.columns[0])

In [0]:
# Union all the selected columns
merged_df = (top_order_col).union(middle_order_col).union(lower_order_col).union(Allrounder_col).union(fast_bowlers_col)

In [0]:
merged_df.show()


+-------------------+
|        batsmanName|
+-------------------+
|     Jos Buttler(c)|
|         Alex Hales|
|      Rilee Rossouw|
|        Virat Kohli|
|   Suryakumar Yadav|
|     Marcus Stoinis|
|      Sikandar Raza|
|     Curtis Campher|
|   Mitchell Santner|
|  Paul van Meekeren|
|Shaheen Shah Afridi|
|      Anrich Nortje|
+-------------------+

